`all_data` is a list where each element is a dictionary that represents a mask. Each dictionary contains three keys:

* `'segmentation'`: This key maps to a 2D numpy array representing the mask itself. Each element of this array corresponds to a pixel in the mask, with a value of 1 indicating that the pixel is part of the mask and a value of 0 indicating that it's not.

* `'area'`: This key maps to an integer representing the total number of pixels in the mask (i.e., the number of 1s in the 'segmentation' array).

* `'label'`: This key maps to a string indicating the type of the mask. It can take on two possible values: 'ballast' or 'non_ballast', depending on which type of object the mask represents. 

For example, if there are N masks in total (regardless of whether they are ballast or non-ballast), `all_data` would be a list of length N. Each element of the list would be a dictionary representing one mask. The mask's segmentation (in the form of a 2D array), area (as an integer), and label (as a string) are accessible via the 'segmentation', 'area', and 'label' keys, respectively.

In [ ]:
import os
import pickle

This part of the code loads `all_data` and transforms it into a dictionary with two fields `'segmentation'` and `'label'`. The label of each are generated automatically depending on the prefix of the folder from which the ballast file is loaded. For example, If the prefix of the folder is 'ballast', then the label of all masks contained in the files under the folder will be 'ballast' and vice verse.

In [ ]:
def load_specific_files(ballast_files, non_ballast_files):
    all_data = []
    for files, label in [(ballast_files, 'ballast'), (non_ballast_files, 'non_ballast')]:
        for filepath in files:
            with open(filepath, 'rb') as f:
                masks = pickle.load(f)
                for mask in masks:
                    mask['label'] = label
                all_data.extend(masks)
    return all_data

# Specify the paths to the specific files you want to process
ballast_files = [
    '/content/drive/MyDrive/Colab Notebooks/ballast_masks_training/surface_1.png_masks.pkl',
    # Add paths to other files as needed...
    '/content/drive/MyDrive/Colab Notebooks/ballast_masks_training/TTC_1.png_masks.pkl',   
    '/content/drive/MyDrive/Colab Notebooks/ballast_masks_training/TTC_4.png_masks.pkl',
    '/content/drive/MyDrive/Colab Notebooks/ballast_masks_training/TTC_5.png_masks.pkl'
]
non_ballast_files = [
    '/content/drive/MyDrive/Colab Notebooks/non_ballast_masks_training/image_10_05.jpg_masks.pkl',
    # Add paths to other files as needed...
    '/content/drive/MyDrive/Colab Notebooks/non_ballast_masks_training/image_11_25.jpg_masks.pkl',
    '/content/drive/MyDrive/Colab Notebooks/non_ballast_masks_training/image_22.jpg_masks.pkl',
    '/content/drive/MyDrive/Colab Notebooks/non_ballast_masks_training/image_25.jpg_masks.pkl',
    '/content/drive/MyDrive/Colab Notebooks/non_ballast_masks_training/image_30.jpg_masks.pkl',
]

training_data = load_specific_files(ballast_files, non_ballast_files)


Firstly, we'll establish a function that calculates the bounding box encompassing non-zero pixels within a two-dimensional tensor, which corresponds to each individual mask. Subsequent to this, every box will be reformatted to a size of 64x64. Implementing these steps enables us to mitigate any potential influence arising from the area, which serves as a distorting factor.Simultaneously, given that a majority of the area in our mask is devoid of data, this type of positioning can help our model concentrate more effectively on regions with non-zero values.

In [ ]:
import numpy as np
def compute_bounding_box(mask):
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)
    rmin, rmax = np.where(rows)[0][[0, -1]]
    cmin, cmax = np.where(cols)[0][[0, -1]]

    return rmin, rmax, cmin, cmax


This function loops over your data, extracts the segmentation masks and labels, converts the masks to tensors, reshapes the tensors to add an extra dimension for the grayscale channel, and pairs each tensor with its corresponding label in a tuple.

In [ ]:
import torch
import cv2  # for resizing images

def to_tensor(data):
    tensor_data = []
    for item in data:
        mask = item['segmentation']
        label = 1 if item['label'] == 'ballast' else 0  # Convert labels to binary values

        # Compute the bounding box around non-zero pixels
        rmin, rmax, cmin, cmax = compute_bounding_box(mask)

        # Extract the ROI from the mask
        roi = mask[rmin:rmax+1, cmin:cmax+1]

        # Resize the ROI to the standard input size for your network
        roi_resized = cv2.resize(roi.astype('float32'), (64, 64))

        # Convert the numpy array to a PyTorch tensor
        roi_tensor = torch.Tensor(roi_resized)

        # Add an extra dimension for the single channel (grayscale)
        roi_tensor = roi_tensor.unsqueeze(0)

        tensor_data.append((roi_tensor, label))

    return tensor_data




Once you've converted your data to tensors, you can use PyTorch's DataLoader class to handle batching and shuffling of the data:

In [ ]:
from torch.utils.data import DataLoader

tensor_data = to_tensor(training_data)



Fantastic! We are ready to begin training your convolutional neural network model on the processed and prepared dataset.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# Model definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.bn1 = nn.BatchNorm2d(32)  # BatchNorm for first Conv layer
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.bn2 = nn.BatchNorm2d(64)  # BatchNorm for second Conv layer
        self.dropout1 = nn.Dropout2d(0.1)
        self.dropout2 = nn.Dropout2d(0.25)
        
        # Automatically calculate size
        x = torch.randn(64,64).view(-1,1,64,64)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 128)
        self.fc2 = nn.Linear(128, 2)

    def convs(self, x):
        x = F.relu(self.bn1(self.conv1(x)))  # Apply BatchNorm after first Conv layer
        x = F.max_pool2d(x, (2, 2))
        x = F.relu(self.bn2(self.conv2(x)))  # Apply BatchNorm after second Conv layer
        x = F.max_pool2d(x, (2, 2))
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)  # Use Dropout after the first fully connected layer
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)




# Training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Net().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)


# Split your data into training and validation sets
# Here, replace dataset with your actual dataset name
train_size = int(0.8 * len(tensor_data))
val_size = len(tensor_data) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(tensor_data, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

n_epochs = 200
patience = 40
min_val_loss = np.inf
counter = 0

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_loss = running_loss / len(train_loader)

    model.eval()
    running_val_loss = 0.0
    with torch.no_grad():
        for i, data in enumerate(val_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

    val_loss = running_val_loss / len(val_loader)
    print(f'Epoch {epoch + 1}, Train loss: {train_loss}, Val loss: {val_loss}')

    # Early stopping
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        print(f'EarlyStopping counter: {counter} out of {patience}')
        if counter >= patience:
            print('Early stopping')
            break
    scheduler.step()



Epoch 1, Train loss: 2.6661027936966386, Val loss: 0.5607708856463433
Epoch 2, Train loss: 0.4499142158341098, Val loss: 0.46164959371089936
Epoch 3, Train loss: 0.4381609421658826, Val loss: 0.44669626504182813
Epoch 4, Train loss: 0.43311976980079303, Val loss: 0.5641560466960073
EarlyStopping counter: 1 out of 40
Epoch 5, Train loss: 0.42570113129430004, Val loss: 0.4618702784180641
EarlyStopping counter: 2 out of 40
Epoch 6, Train loss: 0.41554828201021465, Val loss: 0.7070583656430245
EarlyStopping counter: 3 out of 40
Epoch 7, Train loss: 0.4225700729853147, Val loss: 0.5356265634298325
EarlyStopping counter: 4 out of 40
Epoch 8, Train loss: 0.4228527994899007, Val loss: 0.44764395290985703
EarlyStopping counter: 5 out of 40
Epoch 9, Train loss: 0.4176855453036048, Val loss: 0.4795678235590458
EarlyStopping counter: 6 out of 40
Epoch 10, Train loss: 0.41109686412594537, Val loss: 0.7032594518037513
EarlyStopping counter: 7 out of 40
Epoch 11, Train loss: 0.40467613606483904, Val 

Now we turn to assessing the performance of our model.

In [ ]:
# Specify the paths to the specific test files you want to process
test_ballast_files = [
    '/content/drive/MyDrive/Colab Notebooks/ballast_masks_training/TTC_2.png_masks.pkl', 
    # Add paths to other test ballast files as needed...
]
test_non_ballast_files = [
    '/content/drive/MyDrive/Colab Notebooks/non_ballast_masks_training/image_13_14.jpg_masks.pkl',
    # Add paths to other test non ballast files as needed...
]

# Load the test data
test_data = load_specific_files(test_ballast_files, test_non_ballast_files)

# Convert the test data to tensor format
test_tensor_data = to_tensor(test_data)


In [ ]:
from torch.utils.data import TensorDataset


In [ ]:
# Create a DataLoader for your test data
test_dataset = TensorDataset(torch.stack([sample[0] for sample in test_tensor_data]),
                             torch.Tensor([sample[1] for sample in test_tensor_data]))

test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy of the model on the test images: {100 * correct / total}%')


Test Accuracy of the model on the test images: 81.52173913043478%


Now save our model as "model_001"

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/model_trained/model_001.pt')


Try whether we can load our model successfully

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

# Model definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.bn1 = nn.BatchNorm2d(32)  # BatchNorm for first Conv layer
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.bn2 = nn.BatchNorm2d(64)  # BatchNorm for second Conv layer
        self.dropout1 = nn.Dropout2d(0.1)
        self.dropout2 = nn.Dropout2d(0.25)
        
        # Automatically calculate size
        x = torch.randn(64,64).view(-1,1,64,64)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 128)
        self.fc2 = nn.Linear(128, 2)

    def convs(self, x):
        x = F.relu(self.bn1(self.conv1(x)))  # Apply BatchNorm after first Conv layer
        x = F.max_pool2d(x, (2, 2))
        x = F.relu(self.bn2(self.conv2(x)))  # Apply BatchNorm after second Conv layer
        x = F.max_pool2d(x, (2, 2))
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)  # Use Dropout after the first fully connected layer
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

With the classifier now trained and stored, we are prepared to reload it for the purpose of predicting labels.

In [ ]:
# You need to first create the model object
model = Net()
model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/model_trained/model_001.pt'))


<All keys matched successfully>

In [ ]:
def predict(segmentation, model):
    # Compute the bounding box around non-zero pixels
    rmin, rmax, cmin, cmax = compute_bounding_box(segmentation)

    # Extract the ROI from the mask
    roi = segmentation[rmin:rmax+1, cmin:cmax+1]

    # Resize the ROI to the standard input size for your network
    roi_resized = cv2.resize(roi.astype('float32'), (64, 64))

    # Convert the numpy array to a PyTorch tensor
    roi_tensor = torch.Tensor(roi_resized)

    # Add an extra dimension for the single channel (grayscale)
    roi_tensor = roi_tensor.unsqueeze(0)
    
    # Adding a batch dimension
    roi_tensor = roi_tensor.unsqueeze(0)

    # Transfer to device
    roi_tensor = roi_tensor.to(device)

    # Move the model to the same device
    model = model.to(device)

    # Make prediction
    model.eval()
    with torch.no_grad():
        outputs = model(roi_tensor)
        _, predicted = torch.max(outputs.data, 1)

    # Return True if the model predicts the mask to be a ballast, False otherwise
    return predicted.item() == 1



def filter_masks(masks, model):
    output_masks = []
    for mask in masks:
        if predict(mask['segmentation'], model):
            output_masks.append(mask)
    return output_masks



This function, provided by Meta, is utilized to overlay the masks on the segmented image for visualization.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

Here comes the firsr test case, we want to chenk whether it can successfully filter out the non-ballasts.

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/samples/12.png')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
plt.axis('off')
plt.show()

Output hidden; open in https://colab.research.google.com to view.

Some commands necessary to install dependencys as well as Sam model.

In [ ]:
from IPython.display import display, HTML
display(HTML(
"""
<a target="_blank" href="https://colab.research.google.com/github/facebookresearch/segment-anything/blob/main/notebooks/automatic_mask_generator_example.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>
"""
))

In [ ]:
using_colab = True

In [ ]:
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'
    
    !mkdir images
    !wget -P images https://raw.githubusercontent.com/facebookresearch/segment-anything/main/notebooks/images/dog.jpg
        
    !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth

PyTorch version: 2.0.1+cu118
Torchvision version: 0.15.2+cu118
CUDA is available: True
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-k9yi0ba9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-k9yi0ba9
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e27956588
  Preparing metadata (setup.py) ... done
  Created wheel for segment-anything: filename=segment_anything-1.0-py3-none-any.whl size=36589 sha256=37a476162b9fe54bb407cb08292ea7e577a16b2d73641c92c00c71d95c133bb6
  Stored in directory: /tmp/pip-ephem-wheel-cache-c9w35tlp/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segm

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
# sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

Meta provides an very concise API to segment everything in the image automatically.

In [ ]:
masks_demo = mask_generator.generate(image)

In [ ]:
print(len(masks_demo))

528


In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks)
plt.axis('off')
plt.show() 

Output hidden; open in https://colab.research.google.com to view.

Now see what will happen after filtering the non-ballast objects.

In [ ]:
filtered_masks_demo = filter_masks(masks_demo, model)


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [ ]:
print(len(filtered_masks_demo))

527



As anticipated, the classifier successfully eliminated the twigs located at the top-right and bottom-left corners.

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(filtered_masks)
plt.axis('off')
plt.show() 

Output hidden; open in https://colab.research.google.com to view.

Another test on labeled image, which is annotated by myself.

In [ ]:
image2 = cv2.imread('/content/drive/MyDrive/Colab Notebooks/ballasts_img/surface_1.png')
image2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
plt.axis('off')
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
# sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
masks2 = mask_generator.generate(image2)

In [ ]:
print(len(masks2))

519


In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(masks2)
plt.axis('off')
plt.show() 

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
filtered_masks2 = filter_masks(masks2, model)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [ ]:
print(len(filtered_masks2))

516


From our observations, even in the absence of twigs in the images, the downstream filter manages to retain the majority of the ballast, with only three instances erroneously removed. Therefore, we can infer that the classifier is functioning effectively.

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image)
show_anns(filtered_masks)
plt.axis('off')
plt.show() 

Output hidden; open in https://colab.research.google.com to view.

Now we need implement metrices to help determine the accuracy of out approach compared to the groud truth.

Step 1: Load the JSON data.
You need to open the JSON file and load the data from it:

In [ ]:
import json

# Load the reference data
with open('/content/drive/MyDrive/Colab Notebooks/ballasts_img/surface_1_png.rf.e373bd37e96d074824edb4c4d9296655.json', 'r') as f:
    reference_data = json.load(f)


Step 2: Convert the reference data to polygons.
Here, we'll iterate through the reference data and convert each region to a polygon. Let's use the Polygon class from the shapely.geometry module:

In [ ]:
from shapely.geometry import Polygon

# Convert the reference data to polygons
reference_polygons = []
for entry in reference_data.values():
    for region in entry['regions']:
        shape_attributes = region['shape_attributes']
        x_points = shape_attributes['all_points_x']
        y_points = shape_attributes['all_points_y']
        polygon = Polygon(zip(x_points, y_points))
        reference_polygons.append(polygon)


Step 3: Find the exterior of the predicted mask and convert it to a polygon. Here, we'll use the findContours function from the OpenCV library to find the exterior of the mask:

In [ ]:
import cv2
import numpy as np



predicted_polygons = []

for mask in masks:
    # Convert mask['segmentation'] to a proper binary image
    binary_mask = np.array(mask['segmentation'], dtype=np.uint8)

    # Find the contours of the mask
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convert the contours to polygons
    for contour in contours:
        if len(contour) > 1:
            polygon = Polygon(np.squeeze(contour))
            predicted_polygons.append(polygon)




Step 4: Calculate the IoU for the predicted polygons and the reference polygons:

In [ ]:
# Initialize counters for true positives, false positives, and false negatives
tp, fp, fn = 0, 0, 0

# Threshold for IoU
iou_threshold = 0.5

for predicted_polygon in predicted_polygons:
    # Check if the predicted polygon matches any of the reference polygons
    if any(predicted_polygon.intersection(reference_polygon).area / predicted_polygon.union(reference_polygon).area > iou_threshold for reference_polygon in reference_polygons):
        tp += 1
    else:
        fp += 1

# Count the reference polygons that don't match any predicted polygons
for reference_polygon in reference_polygons:
    if not any(predicted_polygon.intersection(reference_polygon).area / predicted_polygon.union(reference_polygon).area > iou_threshold for predicted_polygon in predicted_polygons):
        fn += 1


/usr/local/lib/python3.10/dist-packages/shapely/set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)


Step 5: Calculate precision, recall, and F1-score:

In [ ]:
# Calculate precision, recall, and F1-score
precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f'Precision: {precision}, Recall: {recall}, F1-score: {f1_score}')


Precision: 0.8336314847942755, Recall: 0.8910133843212237, F1-score: 0.8613678373382625


In [ ]:
filtered_polygons = []

for mask in filtered_masks:
    # Convert mask['segmentation'] to a proper binary image
    binary_mask = np.array(mask['segmentation'], dtype=np.uint8)

    # Find the contours of the mask
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Convert the contours to polygons
    for contour in contours:
        if len(contour) > 1:
            polygon = Polygon(np.squeeze(contour))
            filtered_polygons.append(polygon)


In [ ]:
# Initialize counters for true positives, false positives, and false negatives
tp, fp, fn = 0, 0, 0

# Threshold for IoU
iou_threshold = 0.5

# For each filtered (predicted) polygon...
for filtered_polygon in filtered_polygons:
    # Check if the filtered polygon matches any of the reference polygons
    if any(filtered_polygon.intersection(reference_polygon).area / filtered_polygon.union(reference_polygon).area > iou_threshold for reference_polygon in reference_polygons):
        tp += 1
    else:
        fp += 1

# Count the reference polygons that don't match any filtered polygons
for reference_polygon in reference_polygons:
    if not any(filtered_polygon.intersection(reference_polygon).area / filtered_polygon.union(reference_polygon).area > iou_threshold for filtered_polygon in filtered_polygons):
        fn += 1

# Calculate precision, recall, and F1-score
precision = tp / (tp + fp) if tp + fp > 0 else 0
recall = tp / (tp + fn) if tp + fn > 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

print(f'Precision: {precision}, Recall: {recall}, F1-score: {f1_score}')



Precision: 0.8333333333333334, Recall: 0.8891013384321224, F1-score: 0.8603145235892692
